In [21]:
import cv2
import mediapipe as mp
import pandas as pd

In [2]:
# Khởi tạo MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, smooth_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

In [5]:
def extract_parts_of_file(image_path):
    """
    Trích xuất ra các phần của 1 
    """
    file_name, file_extension = image_path.split(".")
    return file_name, file_extension

In [ ]:

def extract_key_points_from_image(image_path):
    # Lấy ra các phần của một 
    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Phát hiện pose
    results = pose.process(image_rgb)
    print(results)

    # Lưu thông tin của từng keypoint vào danh sách
    key_points_data = []
    if results.pose_landmarks:
        for id, landmark in enumerate(results.pose_landmarks.landmark):
            key_points_data.append(
                {
                    "part": mp_pose.PoseLandmark(id).name,  # Lấy tên của keypoint
                    "x": landmark.x * image_width,  # Chuyển đổi tọa độ chuẩn hóa về pixel
                    "y": landmark.y * image_height,  # Chuyển đổi tọa độ chuẩn hóa về pixel
                    "z": landmark.z * image_width,
                    "visibility": landmark.visibility,
                }
            )

        # Vẽ keypoints lên ảnh
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Tạo DataFrame từ dữ liệu và lưu vào CSV
    df_keypoints = pd.DataFrame(key_points_data)
    df_keypoints.to_csv("pose_keypoints.csv", index=False)

    # Lưu ảnh đã vẽ keypoints vào file mới
    cv2.imwrite("pose_image.png", image)

    # Kết thúc MediaPipe Pose
    pose.close()

In [ ]:
# Test trích xuất ảnh
